In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from multipledispatch import dispatch

In [2]:
pkg_cnt = [1000, 1000, 1000]
cargo_distribution = [.5, .4, .1]
weight = [43, 48, 7.1]

container_dim = [11800, 2340, 2690]
volume = np.prod(container_dim, dtype = "double")
weight_limit = 22_000

space_occupancy = .05

c1_dim1 = [605, 470, 500]
c1_dim2 = [470, 605, 500]
c1_vol = np.prod(c1_dim1)
c2_dim1 = [675, 490, 510]
c2_dim2 = [490, 675, 510]
c2_vol = np.prod(c2_dim1)
c3_dim1 = [1150, 290, 230]
c3_dim2 = [290, 1150, 230]
c3_vol = np.prod(c3_dim1)

box_dims = np.array([c1_dim1, c2_dim1, c3_dim1])

In [3]:
def vol_limit(container_dim = container_dim, cont_dims = container_dim, c1_dim1 = c1_dim1, c2_dim2 = c2_dim2, c3_dim1 = c3_dim1, occupancy = .9,
             weight_limit = 22_000):
    pkg_cnt = cargo_count()
#     for j in np.arange(np.sum(pkg_cnt)):
    return pkg_cnt

In [4]:
def cargo_count(container_dim = container_dim, box_dims = box_dims, occupancy = space_occupancy,
               cargo_distribution = cargo_distribution):
    cont_vol = np.prod(container_dim, dtype = "double")
    volumes = list()
    for box in box_dims:
        volumes.append(np.prod(box))
    cont_vol_occ = cont_vol * occupancy
    vol_allocation = np.array(cargo_distribution) * cont_vol_occ
    cargo_count = np.divide(vol_allocation, volumes)
    return np.trunc(cargo_count)

In [5]:
num_boxes = cargo_count()
num_boxes

array([13.,  8.,  4.])

In [6]:
class Point():
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
    @classmethod
    def two_dim_point(cls, x, y):
        return cls(x, y, 0)
    
    def __str__(self):
        return "(" + str(self.x) + "," + str(self.y) + "," + str(self.z) + ")"
    
    def __gt__(self, Point2):
        return np.linalg.norm([self.x, self.y, self.z]) > np.linalg.norm([Point2.x, Point2.y, Point2.z])
    
#     def __eq__(self, Point2):
#         return ((self.x == Point2.x) and (self.y == Point2.y))
    

In [7]:
class Box():
    def __init__(self, btype, orientation, dims):
        self.btype = btype
        if orientation == 1:
            hold = dims[0]
            dims[0] = dims[1]
            dims[1] = hold
        self.orientation = orientation
        self.dims = dims
        
    def flip(self):
        hold = self.dims[0]
        self.dims[0] = self.dims[1]
        self.dims[1] = hold

In [11]:
# corner: list of corner points. (Use Point object defined above)
# space: No clue. It's just there. Don't question it.
# box_count: number of boxes in container.
# location of boxes: Set as 

class Container(object):
    def __init__(self, corner, space, box_count, box_loc, dimension):
        self.corner = corner
        self.space = space
        self.box_count = box_count
        self.box_loc = box_loc
        self.dimension = dimension
        self.ordered_box = []
        self.corner_3d = []
        self.corner_3d_prev = []
        self.k_star = 0
        self.t_sorted = []
        self.avail_box_list = []
        
    @classmethod
    def empty(cls, dimension):
        return cls([], 0, 0, {}, dimension)
    
    # box_type: the box number of the box_type you are placing in box
    # orientation: orientation number corresponding to the following convention:
        # 0: (L, W, H) Note: This is the original input sequence.
        # 1: (W, L, H)
    # corner: An item of Point class with field values set for the x, y, and z values.
    # dims: A list with the dimension of boxes. 
    # cargo: box object
    
    #This class method acts to "place" a box in the container.    
    def place(self, corner, cargo, box_list = num_boxes, box_dims = box_dims):
        
        dictionary = self.box_loc.items()
        val = -1        
        self.box_loc[corner] = (cargo.btype, corner.x + cargo.dims[0], corner.y + cargo.dims[1], 
                           corner.z + cargo.dims[2])
        
        self.find_corners_2d(box_list, box_dims)
        
        val = self.eval_2d()
        
        self.box_loc.pop(corner, None)
        
#         print("after popping box")
        
        cargo.flip()
        
        self.box_loc[corner] = (cargo.btype, corner.x + cargo.dims[0], corner.y + cargo.dims[1], 
                                corner.z + cargo.dims[2])
        
        self.find_corners_2d(box_list, box_dims)
        
        val1 = self.eval_2d()
        
        self.box_loc.pop(corner, None)
        if (val1 > val):
            cargo.flip()
        self.box_loc[corner] = (cargo.btype, corner.x + cargo.dims[0], corner.y + cargo.dims[1], 
                                corner.z + cargo.dims[2])
        self.box_count += 1
        
    def place_3d(self, corner, cargo, box_list = num_boxes, box_dims = box_dims):
        
        dictionary = self.box_loc.items()
        val = -1        
        self.box_loc[corner] = (cargo.btype, corner.x + cargo.dims[0], corner.y + cargo.dims[1], 
                           corner.z + cargo.dims[2])
        
        self.find_corners_3d(box_list, box_dims)
        
        val = self.eval_3d(box_list, box_dims)
        
        self.box_loc.pop(corner, None)
        
#         print("after popping box")
        
        cargo.flip()
        
        self.box_loc[corner] = (cargo.btype, corner.x + cargo.dims[0], corner.y + cargo.dims[1], 
                                corner.z + cargo.dims[2])
        
        self.find_corners_3d(box_list, box_dims)
        
        val1 = self.eval_3d(box_list, box_dims)
        
        self.box_loc.pop(corner, None)
        if (val1 > val):
            cargo.flip()
        self.box_loc[corner] = (cargo.btype, corner.x + cargo.dims[0], corner.y + cargo.dims[1], 
                                corner.z + cargo.dims[2])
        self.box_count += 1
        
        

    
    def order_boxes(self):
        pairs = self.box_loc.items()
        
        return sorted(pairs, key = lambda x: (x[1][2], x[1][1]), reverse = True)
    
    def set_corner(self, point):
        self.corner.append(point)
            
        
    # box_list: List of boxes not yet added to container
    
    def sort_corners(self):
        return self.corner.sort()
    
    @dispatch(np.ndarray, np.ndarray)
    def find_corners_2d(self, box_list, box_dims):
        self.corner = []
        if self.box_count == 0:
            self.corner.append(Point.two_dim_point(0, 0))
            return
        #Phase 1: Identify Extreme Points.
        
        x_bar = 0
        m = 0
        em = list()
        em.append(0)
        ordered_boxes = self.order_boxes()
        self.ordered_box = ordered_boxes
        
        for index, value in enumerate(ordered_boxes):
            corner, b_data = value
            btype, x, y, z = b_data
            if x > x_bar:
                m += 1
                em.append(index)
                x_bar = x
        # Phase 2: Find Corner Points
        self.corner.append(Point.two_dim_point(0, ordered_boxes[0][1][2]))
        self.corner.append(Point.two_dim_point(ordered_boxes[em[m]][1][1], 0))
        for j in np.arange(2, m+1):
            self.corner.append(Point.two_dim_point(ordered_boxes[em[j-1]][1][1], ordered_boxes[em[j]][1][2]))
            
        # Phase 3: Remove redundant/infeasible corner points:
        pres = []
        for i in np.arange(len(box_list)):
            if box_list[i] != 0:
                pres.append(box_dims[i])
        pres = np.array(pres)
        x_avail = pres[:, 0]
        y_avail = pres[:, 1]
        to_be_removed = []
        
        for corn in self.corner:
            if (corn.x + min(x_avail) > self.dimension[0]) or (corn.y + min(y_avail) > self.dimension[1]):
                to_be_removed.append(corn)
        self.corner = [e for e in self.corner if e not in to_be_removed]
        
    @dispatch()
    def eval_2d(self):
        if(len(self.corner) > 0):
            sum = self.corner[0].x*self.dimension[1]
            for i in np.arange(1, len(self.corner)):
                sum+= (self.corner[i].x - self.corner[i - 1].x) * self.corner[i - 1].y
            sum += + (self.dimension[0] - self.corner[len(self.corner) -1].x) * self.corner[len(self.corner) -1].y
            return sum
        if(len(self.corner) == 1):
            return self.dimension[0] * self.dimension[1]
    
    @dispatch(list)
    def eval_2d(self, available_boxes, box_list, box_dims):
        self.find_corners_2d(box_list, box_dims, available_boxes)
        if(len(self.corner) > 0):
            sum = self.corner[0].x*self.dimension[1]
            for i in np.arange(1, len(self.corner)):
                sum+= (self.corner[i].x - self.corner[i - 1].x) * self.corner[i - 1].y
            sum += + (self.dimension[0] - self.corner[len(self.corner) -1].x) * self.corner[len(self.corner) -1].y
            return sum
        if(len(self.corner) == 1):
            return self.dimension[0] * self.dimension[1]
        
    def eval_3d(self, box_list, box_dims):
        sum = 0
        for i in np.arange(2, self.k_star):
            sum += (self.t_sorted[i] - self.t_sorted[i - 1]) * eval_2d(self.avail_box_list[i], box_list, box_dims)
        print("k_star", self.k_star)
        print("length of t", len(self.t_sorted))
        if(self.k_star < len(self.t_sorted)):
            sum += (self.dimension[2] - self.t_sorted[self.k_star]) * eval_2d(self.avail_box_list[self.k_star], box_list, box_dims)
        return sum
    
    @dispatch(np.ndarray, np.ndarray, list)
    def find_corners_2d(self, box_list, box_dims, available_boxes):
        self.corner = []
        if self.box_count == 0:
            self.corner.append(Point.two_dim_point(0, 0))
            return
        #Phase 1: Identify Extreme Points.
        
        x_bar = 0
        m = 0
        em = list()
        em.append(0)
        ordered_boxes = sorted(available_boxes, key = lambda x: (x[1][2], x[1][1]), reverse = True)
        
        for index, value in enumerate(ordered_boxes):
            corner, b_data = value
            btype, x, y, z = b_data
            if x > x_bar:
                m += 1
                em.append(index)
                x_bar = x
        # Phase 2: Find Corner Points
        print("ordered_boxes: ", ordered_boxes)
        self.corner.append(Point.two_dim_point(0, ordered_boxes[0][1][2]))
        self.corner.append(Point.two_dim_point(ordered_boxes[em[m]][1][1], 0))
        for j in np.arange(2, m+1):
            self.corner.append(Point.two_dim_point(ordered_boxes[em[j-1]][1][1], ordered_boxes[em[j]][1][2]))
            
        # Phase 3: Remove redundant/infeasible corner points:
        pres = []
        for i in np.arange(len(box_list)):
            if box_list[i] != 0:
                pres.append(box_dims[i])
        pres = np.array(pres)
        x_avail = pres[:, 0]
        y_avail = pres[:, 1]
        to_be_removed = []
        
        for corn in self.corner:
            if (corn.x + min(x_avail) > self.dimension[0]) or (corn.y + min(y_avail) > self.dimension[1]):
                to_be_removed.append(corn)
        self.corner = [e for e in self.corner if e not in to_be_removed]

    
    
# Find 3d Corners:

    def find_corners_3d(self, box_list, box_dims):
        self.corner_3d = []
        if self.box_count == 0:
            self.corner_3d.append(Point(0, 0, 0))
            return
        ordered_boxes = self.order_boxes()
        self.ordered_box = ordered_boxes
        T = set()
        T.add(0)
        for index, value in enumerate(ordered_boxes):
            corner, b_data = value
            btype, x, y, z = b_data
            T.add(z)
        present = []
        for i in np.arange(len(box_list)):
            if box_list[i] != 0:
                present.append(box_dims[i])
        T_sorted = sorted(T)
#         print("T_sorted", T_sorted)
        present = np.array(present)
        z_avail = present[:, 2]
        k = 1
        r = len(T_sorted)
        self.t_sorted = T_sorted
#         self.avail_box_list.append([(Point(-1,-1,-1), (-1, -1, -1))])
        while(k <= r) and (T_sorted[k-1] + min(z_avail) <= self.dimension[2]):
            Ik = []
            print("run", k)
            for i in ordered_boxes:
#                 print("i[1][3]:", i[1][3])
#                 print("T_sorted[k-1]:", T_sorted[k-1])
                if(i[1][3] > T_sorted[k-1]):
                    Ik.append(i)
            if(len(Ik) != 0):
                self.find_corners_2d(box_list, box_dims, Ik)
                self.avail_box_list.append(Ik)
            else:
                self.corner.append(Point(0, 0, T_sorted[k-1]))
            under_consideration = []
            for i in self.corner:
#                 print("run " + str(k) + ": corner " + str(i) + ": Height: " + str(T_sorted[k-1]) )
                under_consideration.append(Point(i.x, i.y, T_sorted[k-1]))
                
            
            for pt in under_consideration:
                check = 0
                print("point being considered:",pt)
                for prev in self.corner_3d_prev:
                    if (pt.x == prev.x and pt.y == prev.y):
                        check = 1
                if (check == 0):
                    self.corner_3d.append(pt)
            self.k_star = k
            k = k + 1
            self.corner_3d_prev = self.corner_3d
            print("current corners")
            for i in self.corner_3d:
                print(i)
        self.corner_3d_prev = []
        check = set()
        for i in self.corner_3d:
            check.add(i)
        self.corner_3d = sorted(check)

# Testing

In [12]:
cont_3d = Container.empty([11800, 2340, 2690])

# for i in np.arange(num_boxes[0]):
#     print("run", i)
cargo = Box(0, 0, box_dims[0])
cont_3d.find_corners_3d(num_boxes, box_dims)
print("box 1 added")
cont_3d.place_3d(cont_3d.corner_3d[0], cargo)
num_boxes[0] -= 1
cont_3d.find_corners_3d(num_boxes, box_dims)
print("box 2 added")
cont_3d.place_3d(cont_3d.corner_3d[2], cargo)
num_boxes[0] -= 1
cont_3d.find_corners_3d(num_boxes, box_dims)
print("box 3 added")
cont_3d.place_3d(cont_3d.corner_3d[2], cargo)
num_boxes[0] -= 1
cont_3d.find_corners_3d(num_boxes, box_dims)

box 1 added
k_star 0
length of t 0
k_star 0
length of t 0
run 1
ordered_boxes:  [(<__main__.Point object at 0x00000236518C3520>, (0, 470, 605, 500))]
point being considered: (0,605,0)
point being considered: (470,0,0)
current corners
(0,605,0)
(470,0,0)
run 2
point being considered: (0,605,500)
point being considered: (470,0,500)
point being considered: (0,0,500)
current corners
(0,605,0)
(470,0,0)
(0,0,500)
box 2 added
run 1
ordered_boxes:  [(<__main__.Point object at 0x00000236518C3C40>, (0, 470, 1210, 500)), (<__main__.Point object at 0x00000236518C3520>, (0, 470, 605, 500))]
point being considered: (0,1210,0)
point being considered: (470,0,0)
current corners
(0,1210,0)
(470,0,0)
run 2
point being considered: (0,1210,500)
point being considered: (470,0,500)
point being considered: (0,0,500)
current corners
(0,1210,0)
(470,0,0)
(0,0,500)
k_star 2
length of t 2
run 1
ordered_boxes:  [(<__main__.Point object at 0x00000236518C3C40>, (0, 605, 1075, 500)), (<__main__.Point object at 0x000

In [32]:
cont_3d.find_corners_3d(num_boxes, box_dims)
    
print("printing corners")
for i in cont_3d.corner_3d:
    print(i)

run 1
ordered_boxes:  [(<__main__.Point object at 0x0000013778D6C2B0>, (0, 470, 1815, 500)), (<__main__.Point object at 0x0000013778D4D7C0>, (0, 470, 1210, 500)), (<__main__.Point object at 0x0000013778D4D550>, (0, 470, 605, 500))]
point being considered: (0,1815,0)
point being considered: (470,0,0)
current corners
(0,1815,0)
(470,0,0)
run 2
point being considered: (0,1815,500)
point being considered: (470,0,500)
point being considered: (0,0,500)
current corners
(0,1815,0)
(470,0,0)
(0,0,500)
printing corners
(470,0,0)
(0,0,500)
(0,1815,0)


In [33]:
cont_3d.avail_box_list

[[(<__main__.Point at 0x13778d4d550>, (0, 470, 605, 500))],
 [(<__main__.Point at 0x13778d4d7c0>, (0, 470, 1210, 500)),
  (<__main__.Point at 0x13778d4d550>, (0, 470, 605, 500))],
 [(<__main__.Point at 0x13778d6c2b0>, (0, 470, 1815, 500)),
  (<__main__.Point at 0x13778d4d7c0>, (0, 470, 1210, 500)),
  (<__main__.Point at 0x13778d4d550>, (0, 470, 605, 500))],
 [(<__main__.Point at 0x13778d6c2b0>, (0, 470, 1815, 500)),
  (<__main__.Point at 0x13778d4d7c0>, (0, 470, 1210, 500)),
  (<__main__.Point at 0x13778d4d550>, (0, 470, 605, 500))]]

In [112]:
cont_2d = Container.empty([11800, 2340, 2690])

In [13]:
cargo = Box(0, 0, box_dims[0])
cargo.dims
cargo1 = Box(0, 0, box_dims[0])

In [23]:
cont_2d.place(Point.two_dim_point(0,0), cargo)

cont_2d.box_loc

cont_2d.find_corners_2d(num_boxes, box_dims)

print(cont_2d.corner)

[<__main__.Point object at 0x0000020C841DF730>, <__main__.Point object at 0x0000020C841E4700>]


In [156]:
for i in cont_2d.corner:
    print(i)

(0,470,0)
(605,0,0)


In [157]:
cont_2d.place(cont_2d.corner[0], cargo1)
cont_2d.find_corners_2d(num_boxes, box_dims)
for i in cont_2d.corner:
    print(i)

0
1
em: [0, 0, 1]
m: 2
(0,940,0)
(605,0,0)
(605,470,0)


In [290]:
cont_2d.place(cont_2d.corner[1], cargo1)
cont_2d.find_corners_2d(num_boxes, box_dims)
for i in cont_2d.corner:
    print(i)

cont_2d.sort_corners()

for i in cont_2d.corner:
    print(i)

(0,940,0)
(1210,0,0)
(605,470,0)
False
False
(0,940,0)
(1210,0,0)
(605,470,0)


In [291]:
cont_2d.find_corners_2d(num_boxes, box_dims)

print(cont_2d.corner)

[<__main__.Point object at 0x000002B8AFACAB20>, <__main__.Point object at 0x000002B8AFACAD90>, <__main__.Point object at 0x000002B8AFABA580>]


In [292]:
for i in cont_2d.corner:
    print(i)

(0,940,0)
(1210,0,0)
(605,470,0)


In [19]:
cont_test = Container.empty([1, 1, 1])
cont_test.find_corners_2d(num_boxes, box_dims)
print(cont_test.corner[0])

(0,0,0)


In [91]:
cont_2d = Container.empty([11800, 2340, 2690])

In [92]:
cont_2d.place(Point.two_dim_point(0, 0), cargo)
cont_2d.find_corners_2d(num_boxes, box_dims)
cont_2d.sort_corners()

cont_2d.place(cont_2d.corner[0], cargo)
cont_2d.find_corners_2d(num_boxes, box_dims)
cont_2d.sort_corners()

cont_2d.place(cont_2d.corner[0], cargo)
cont_2d.find_corners_2d(num_boxes, box_dims)
cont_2d.sort_corners()

cont_2d.place(cont_2d.corner[0], cargo)
cont_2d.find_corners_2d(num_boxes, box_dims)
cont_2d.sort_corners()

# cont_2d.place(cont_2d.corner[0], cargo)
# cont_2d.find_corners_2d(num_boxes, box_dims)
# cont_2d.sort_corners()
# cont_2d.place(Point.two_dim_point(605, 470), cargo)
# # cont_2d.place(Point.two_dim_point(1210, 470), cargo)

val 0
val 1 0
m: 1
m: 1
m: 2
val 568700
val 1 5975475
m: 1
m, x_bar (1, 605)
box_locations {<__main__.Point object at 0x00000181D6AB7550>: (0, 605, 470, 500), <__main__.Point object at 0x00000181D6C11040>: (0, 605, 940, 500), <__main__.Point object at 0x00000181D6AA6E20>: (0, 1210, 470, 500)}
m, x_bar (2, 1210)
box_locations {<__main__.Point object at 0x00000181D6AB7550>: (0, 605, 470, 500), <__main__.Point object at 0x00000181D6C11040>: (0, 605, 940, 500), <__main__.Point object at 0x00000181D6AA6E20>: (0, 1210, 470, 500)}
m: 2
m, x_bar (1, 605)
box_locations {<__main__.Point object at 0x00000181D6AB7550>: (0, 605, 470, 500), <__main__.Point object at 0x00000181D6C11040>: (0, 605, 940, 500), <__main__.Point object at 0x00000181D6AA6E20>: (0, 1075, 605, 500)}
m, x_bar (2, 1075)
box_locations {<__main__.Point object at 0x00000181D6AB7550>: (0, 605, 470, 500), <__main__.Point object at 0x00000181D6C11040>: (0, 605, 940, 500), <__main__.Point object at 0x00000181D6AA6E20>: (0, 1075, 605, 

In [93]:
cont_2d.find_corners_2d(num_boxes, box_dims)
for i in cont_2d.corner:
    print(i)

print("\n")
cont_2d.sort_corners()

for i in cont_2d.corner:
    print(i)
    


m: 1
(0,940,0)
(1210,0,0)


(0,940,0)
(1210,0,0)


In [94]:
for corner, box in cont_2d.box_loc.items():
    print("Corner: " + str(corner) + "| Box End Location: " + str(box))

Corner: (0,0,0)| Box End Location: (0, 605, 470, 500)
Corner: (0,470,0)| Box End Location: (0, 605, 940, 500)
Corner: (605,0,0)| Box End Location: (0, 1210, 470, 500)
Corner: (605,470,0)| Box End Location: (0, 1210, 940, 500)


In [214]:
print(type(cont_2d.corner))

<class 'list'>


In [123]:
cont_2d.ordered_box[0]

[(<__main__.Point at 0x29f7ae2d850>, (0, 605, 1410, 500)),
 (<__main__.Point at 0x29f7af7b250>, (0, 1210, 940, 500)),
 (<__main__.Point at 0x29f7ae2d4c0>, (0, 605, 940, 500)),
 (<__main__.Point at 0x29f7af7b310>, (0, 1815, 470, 500)),
 (<__main__.Point at 0x29f7ae2d910>, (0, 1210, 470, 500)),
 (<__main__.Point at 0x29f7ae2da90>, (0, 605, 470, 500))]

In [296]:
test = np.arange(9).reshape(3, -1)
test

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [332]:
type(box_dims)

list

In [337]:
box_dims[:,0]

array([ 605,  675, 1150])

In [298]:
arr =[1,0,1]

In [335]:
test[:,0]

array([0, 3, 6])

In [300]:
test

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [301]:
np.arange(2,2)

array([], dtype=int32)

In [36]:
cont_3d.box_loc

{<__main__.Point at 0x1e710d37430>: (0, 470, 605, 500)}

val 0
val 1 0


T_sorted [0, 500]
run 1
i[1][3]:  500
Ik [(<__main__.Point object at 0x000001E710D7CA90>, (0, 470, 605, 500))]
ordered_boxes:  [(<__main__.Point object at 0x000001E710D7CA90>, (0, 470, 605, 500))]


IndexError: list index out of range

(0,605,0)
(470,0,0)


In [47]:
num_boxes

array([17.,  0.,  0.])

In [365]:
for i in cont_2d.corner:
    print(i)

(19360,0,0)
(19360,940,0)


In [424]:
num_boxes

array([ 54., 176.,  77.])

In [18]:
testpt1 = Point(1,2,3)
testpt2 = Point(1,2,4)
testpt3 = Point(1,3,3)

In [188]:
space_occupancy = .05

In [189]:
testpt1 == testpt2

NameError: name 'testpt1' is not defined

In [277]:
num_boxes = cargo_count()
num_boxes

array([  0.,   0., 193.])

In [204]:
cont_2d = Container.empty([11800, 2340, 2690])
cont_2d.find_corners_2d(num_boxes, box_dims)
for i in np.arange(num_boxes[0]):
    cargo = Box(0, 0, box_dims[0])
    print(cont_2d.corner)
    cont_2d.place(cont_2d.corner[0], cargo)
    cont_2d.find_corners_2d(num_boxes, box_dims)
    cont_2d.sort_corners()
    num_boxes[0] -= 1

for i in np.arange(num_boxes[1]):
    cargo = Box(1, 0, box_dims[1])
    cont_2d.place(cont_2d.corner[0], cargo)
    cont_2d.find_corners_2d(num_boxes, box_dims)
    cont_2d.sort_corners()
    num_boxes[1] -= 1
    
for i in np.arange(num_boxes[2]):
    cargo = Box(2, 0, box_dims[2])
    cont_2d.place(cont_2d.corner[0], cargo)
    cont_2d.find_corners_2d(num_boxes, box_dims)
    cont_2d.sort_corners()
    num_boxes[2] -= 1

[<__main__.Point object at 0x000001E710D7C3A0>, <__main__.Point object at 0x000001E710D7C280>]
[<__main__.Point object at 0x000001E710D7C280>, <__main__.Point object at 0x000001E710D7CA60>]
[<__main__.Point object at 0x000001E710D46C10>, <__main__.Point object at 0x000001E710D7C2B0>, <__main__.Point object at 0x000001E710D7CA60>]
[<__main__.Point object at 0x000001E710D7CA60>, <__main__.Point object at 0x000001E710D7C2B0>]
[<__main__.Point object at 0x000001E710D46EB0>, <__main__.Point object at 0x000001E710D7C2B0>, <__main__.Point object at 0x000001E710D46250>]
[<__main__.Point object at 0x000001E710D7C2B0>, <__main__.Point object at 0x000001E710D7C700>, <__main__.Point object at 0x000001E710D46250>, <__main__.Point object at 0x000001E710D7C940>]
[<__main__.Point object at 0x000001E710D46FD0>, <__main__.Point object at 0x000001E710D46E80>, <__main__.Point object at 0x000001E710D46250>, <__main__.Point object at 0x000001E710D7C700>, <__main__.Point object at 0x000001E710D7C940>]
[<__ma

In [205]:
for corner, box in cont_2d.box_loc.items():
    print("Corner: " + str(corner) + "| Box Location: " + str(box))

Corner: (0,0,0)| Box Location: (0, 470, 605, 500)
Corner: (470,0,0)| Box Location: (0, 940, 605, 500)
Corner: (0,605,0)| Box Location: (0, 605, 1075, 500)
Corner: (605,605,0)| Box Location: (0, 1075, 1210, 500)
Corner: (1075,0,0)| Box Location: (0, 1680, 470, 500)
Corner: (1075,470,0)| Box Location: (0, 1545, 1075, 500)
Corner: (0,1210,0)| Box Location: (0, 605, 1680, 500)
Corner: (605,1210,0)| Box Location: (0, 1210, 1680, 500)
Corner: (1545,470,0)| Box Location: (0, 2150, 940, 500)
Corner: (1210,1075,0)| Box Location: (0, 1680, 1680, 500)
Corner: (0,1680,0)| Box Location: (0, 605, 2150, 500)
Corner: (605,1680,0)| Box Location: (0, 1210, 2150, 500)
Corner: (1680,940,0)| Box Location: (0, 2285, 1410, 500)
Corner: (1210,1680,0)| Box Location: (1, 1885, 2170, 510)
Corner: (2285,0,0)| Box Location: (1, 2960, 490, 510)
Corner: (2285,490,0)| Box Location: (1, 2775, 1165, 510)
Corner: (1885,1410,0)| Box Location: (1, 2560, 1900, 510)
Corner: (1885,1900,0)| Box Location: (1, 2375, 2575, 510)


In [280]:
container_dim[0] * container_dim[1]

27612000

In [284]:
21344000/27612000

0.7729972475735187

In [282]:
num_boxes

array([  0.,   0., 129.])

In [164]:
261-183

78

In [283]:
0*box_dims[0][0] * box_dims[0][1] + 0 * box_dims[1][0] * box_dims[1][1] + 64 * box_dims[2][0] * box_dims[2][1]

21344000

In [77]:
cargo.flip()
cargo.dims
cargo.flip()
cargo.btype

1 [470 605 500]
2 [470 605 500]
3 [605 605 500]
4 [605 470 500]
1 [605 470 500]
2 [605 470 500]
3 [470 470 500]
4 [470 605 500]


0

In [71]:
cargo

In [25]:
testpt3 not in [testpt2, testpt2]

True

corner <- associated box type

[{corner(Point): (box_type, end_x, end_y, end_z)}, {corner(Point): (box_type, end_x, end_y, end_z)}, {corner(Point): (box_type, end_x, end_y, end_z)}, {corner(Point): (box_type, end_x, end_y, end_z)}, {corner(Point): (box_type, end_x, end_y, end_z)}]

0: [100, 200, 100]
0b: [200, 100, 100]

(0, 100, 0): (100, 200, 100)